Install necessary libraries

In [ ]:
%pip install \ transformers==4.27.2 \ datasets==2.11.0 \ evaluate==0.4.0 \ rouge_score==0.1.2 \ loralib==0.1.1 \ peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.8 MB/s eta 0:00:00


Import the necessary components

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


### Step 1 - Load dataset : We are going to Dialog Sum dataset from HuggingFace to fine tune our model. This dataset contains 10k + dialogues with the corresponding manually labeled summaries and topics.

In [ ]:
data_set_name = "knkarthick/dialogsum"
dataset = load_dataset(data_set_name)
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [ ]:
# lets take a look at one of the examples. As you can see, each datapoint is a dialogue with its corresponding summary and topic.

dataset['train'][0]

{'id': 'train_0',
 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 'summary': "Mr. Smith'

### Step2: Load the pretrained FLAN-T5 model and its corresponding tokenizer from HuggingFace.

In [ ]:
model_name = "google/flan-t5-base"
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype = torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Step 3: Before finetuning the model, lets use ZERO SHOT inferencing to test model's summarization capabilities.

In [ ]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation between two persons.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
outputs = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens = 150,
    )[0],
    skip_special_tokens = True
)

In [ ]:
print(f'INPUT PROMPT:\n{prompt}')
print("=========================")
print(f'HUMAN SUMMARY:\n{summary}')
print("=========================")
print(f'MODEL OUTPUT:\n{outputs}')
print("=========================")

INPUT PROMPT:

Summarize the following conversation between two persons.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
MODEL OUTPUT:
#Person1#: I'm thinking of upg

In [ ]:
# As you can see model does not do a good job at summarizing the task given to it.

### Step 4: Perform finetuning.
We will convert the diagsum dataset into instruction dataset for the LLM and use it to finetune the model.

We will add a start and end prompt to the tokenizer to prepend an instruction at the start of the dialog and to the start of the summary

Example:

Summarize the following conversation.

{dialogue}

Summary:

{summary}

and then preprocess the prompt response dataset into tokens.

In [ ]:
def preprocess_data(example):
  start_prompt = 'Summarize the following conversation.\n\n'
  end_prompt = '\n\nSummary: '
  prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
  example['input_ids'] = tokenizer(prompt, padding='max_length', truncation=True, return_tensors="pt").input_ids
  example['labels'] = tokenizer(example['summary'], padding='max_length', truncation=True, return_tensors="pt").input_ids
  return example

tokenized_datasets = dataset.map(preprocess_data, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary'])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# lets look at the shape of the dataset
tokenized_datasets['train'].shape

(12460, 2)

### Step 5: Use Hugging Face Trainer to fine tune the original model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We are going to use Rogue_Score to evaluate our model. We are going to use evaluate library to evaluate the rogue score.

In [ ]:
rouge_score = evaluate.load('rouge')

In [ ]:
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=original_model,
)
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[12198,  1635,  1737,  ...,     0,     0,     0],
        [12198,  1635,  1737,  ...,     0,     0,     0]]), 'labels': tensor([[ 1363,     5,  3931,  ...,     0,     0,     0],
        [ 8667, 13156,  1217,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'decoder_input_ids': tensor([[    0,  1363,     5,  ...,     0,     0,     0],
        [    0,  8667, 13156,  ...,     0,     0,     0]])}

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 1
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_name.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-dialsum",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    original_model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/hrangi/flan-t5-base-finetuned-dialsum into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.302700,0.157781,29.176200,10.696000,25.956400,27.107300


TrainOutput(global_step=1558, training_loss=3.3007718479220154, metrics={'train_runtime': 4005.2818, 'train_samples_per_second': 3.111, 'train_steps_per_second': 0.389, 'total_flos': 8667537195663360.0, 'train_loss': 3.3007718479220154, 'epoch': 1.0})

In [ ]:
# Lets now use PEFT to fine tune the model.

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType


lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules = ["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [ ]:
peft_model = get_peft_model(original_model, lora_config)


In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 1
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_name.split("/")[-1]

peft_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-peft-finetuned-dialsum",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)


trainer = Seq2SeqTrainer(
    peft_model,
    peft_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/content/flan-t5-base-peft-finetuned-dialsum is already a clone of https://huggingface.co/hrangi/flan-t5-base-peft-finetuned-dialsum. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.303200,0.158484,28.667300,10.159900,25.222800,26.504600


TrainOutput(global_step=1558, training_loss=3.3012275353016687, metrics={'train_runtime': 4024.5012, 'train_samples_per_second': 3.096, 'train_steps_per_second': 0.387, 'total_flos': 8667537195663360.0, 'train_loss': 3.3012275353016687, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.15848436951637268,
 'eval_rouge1': 28.6673,
 'eval_rouge2': 10.1599,
 'eval_rougeL': 25.2228,
 'eval_rougeLsum': 26.5046,
 'eval_runtime': 129.3733,
 'eval_samples_per_second': 3.865,
 'eval_steps_per_second': 0.487,
 'epoch': 1.0}

In [ ]:
peft_model.push_to_hub("Flan-t5-peft-adapter")

adapter_model.bin:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hrangi/Flan-t5-peft-adapter/commit/ce0de43025adf088c8bf9445b8ad61499b2f3beb', commit_message='Upload model', commit_description='', oid='ce0de43025adf088c8bf9445b8ad61499b2f3beb', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig

peft_model_id = "hrangi/Flan-t5-peft-adapter"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
peft_model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

### Lets manually evaluate the model predictions

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

#original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
#original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print("============")
print(f'BASELINE HUMAN SUMMARY:\n{dialogue}')
print("============")
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print("============")
#print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print("============")
print(f'PEFT MODEL: {peft_model_text_output}')


BASELINE HUMAN SUMMARY:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
PEFT MODEL: #Person1# wants to upgrade his system.


In [ ]:
# As you can see the model is able to get the context and is better than the original model summary.